# This notebook will read in data that contains a placename (e.g. China, Wuhan,etc) and then use the geopy packages to convert that name into coordinates to plot on a map.

In [1]:
import pandas as pd 
import numpy as np
import folium
from tqdm import tqdm, tqdm_notebook
tqdm.pandas(tqdm_notebook)

# need to pip install geopy
from geopy.geocoders import Nominatim 
from geopy.exc import GeocoderTimedOut


C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
# this is the Nextstrain data: https://github.com/nextstrain/ncov
ncov_df=pd.read_csv('D:/GitHub/ncov/data/metadata.tsv',encoding="latin-1",  sep='\t', lineterminator='\r')

In [5]:
ncov_df.head()


,strain,virus,gisaid_epi_isl,genbank_accession,date,region,country,division,division_exposure,location,...,length,host,age,sex,originating_lab,submitting_lab,authors,url,title,date_submitted
0,\n/Belgium/SN-03031/2020,ncov,EPI_ISL_416469,?,2020-03-03,Europe,Belgium,Kessel-Lo,Kessel-Lo,NaN,...,29845.0,Human,?,Male,"KU Leuven, Clinical and Epidemiological Virology","KU Leuven, Clinical and Epidemiological Virology",<NA> et al,https://www.gisaid.org,?,2020-03-22
1,\nAnhui/SZ005/2020,ncov,EPI_ISL_413485,?,2020-01-24,Asia,China,Anhui,Anhui,Suzhou,...,29860.0,Human,?,Male,"Department of microbiology laboratory,Anhui Pr...","Department of microbiology laboratory,Anhui Pr...",<NA> et al,https://www.gisaid.org,?,2020-03-05
2,\nAustralia/NSW01/2020,ncov,EPI_ISL_407893,?,2020-01-24,Oceania,Australia,New South Wales,Hubei,Sydney,...,29782.0,Human,?,Male,Centre for Infectious Diseases and Microbiolog...,NSW Health Pathology - Institute of Clinical P...,<NA> et al,https://www.gisaid.org,?,2020-02-06
3,\nAustralia/NSW02/2020,ncov,EPI_ISL_408976,?,2020-01-22,Oceania,Australia,New South Wales,New South Wales,Sydney,...,29741.0,Human,?,Male,Centre for Infectious Diseases and Microbiolog...,NSW Health Pathology - Institute of Clinical P...,<NA> et al,https://www.gisaid.org,?,2020-02-11
4,\nAustralia/NSW03/2020,ncov,EPI_ISL_408977,?,2020-01-25,Oceania,Australia,New South Wales,New South Wales,Sydney,...,29782.0,Human,?,Female,"Serology, Virology and OTDS Laboratories (SAVi...",NSW Health Pathology - Institute of Clinical P...,<NA> et al,https://www.gisaid.org,?,2020-02-11


In [10]:
# Here is where we pass the df field to geocode. replace the ncov_df["division"] with your dataframe field that contains the placname
# The timeout is necessay becasue you are limited to 30,000 requests per day and you do not want to span the endpoint
tqdm.pandas(tqdm_notebook)
nom=Nominatim(user_agent="youvegotmail2@aol.com", timeout=15)
ncov_df["coordinates"]=ncov_df["division"].progress_apply(nom.geocode)

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


  0%|                                                                                         | 0/1313 [00:00<?, ?it/s]

GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests

In [9]:
# This will split the lat/long returned by the geocoder into seperate fields
ncov_df["lat"]=ncov_df["coordinates"].progress_apply(lambda x: x.latitude if x != None else None)
ncov_df["long"]=ncov_df["coordinates"].progress_apply(lambda x: x.longitude if x != None else None)

KeyError: 'coordinates'

In [ ]:
ncov_df.dropna(inplace=True)

In [ ]:
# A lit is needed for Folium in order to plat them
locations = ncov_df[['lat', 'long']]
locationlist = locations.values.tolist()

In [ ]:
# This is taking the coordinates list and platting them in a Heatmap. 
from folium.plugins import HeatMap
hmap = folium.Map(tiles='CartoDB dark_matter',control_scale = True)
hmap.add_child(HeatMap(locationlist, radius = 16, gradient={0.2: 'blue', .5: 'lime', 1: 'red'}))
# hmap.save('./heatmap.html')

In [ ]:
hmap